# Introduzione

L'environment **anaconda** da utilizzare è **webscraping**.  

# Importazione pacchetti

Importare i vari pacchetti necessari  per eseguire le azioni elencate nell'introduzione:
   - selenium (per navigare sulle pagine web)
   - request (scaricare dati da internet)
   - bs4 (beautiful soup, manipolare gli html)
   - os (navigare nei file del computer)
   - pandas (manipolazione/esportazione dataframes)
   - time (per i time sleep)
   - datetime (gestire i timestamp, ovvero le stringhe contenti date e orari)
   - tqdm (per le barre di progresso) 
   - re (lavorare con regex e le regular expressions)

In [1]:
#%% Pacchetti
## Importazione pacchetti per selenium
# Altre funzionalità di selenium
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.common.action_chains import ActionChains
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
#from selenium.common.exceptions import TimeoutException
#from webdriver_manager.chrome import ChromeDriverManager
#-------##-------##-------##-------##-------##-------##-------##-------##-------##-------#
import requests # Importazione pacchetti per scaricare dati da internet
from bs4 import BeautifulSoup # Importazione pacchetti per gestire HTML
import os # Interagire con i file del pc 
from tqdm import tqdm as tqdm # Importazione pacchetti per barra di progresso
import re # Importazione pacchetti per utilizzare regex e le regular expression
import pandas as pd  # Importazione pacchetti per lavorare con i dataframe
import warnings # Importazione pacchetti per gestire i warnings, ad esempio quelli di pandas
from datetime import datetime# Importazione pacchetti per lavorare con timestamps, orari e date
import numpy as np 

#pd.set_option('display.max_rows', None)

In [2]:
# Directory principale dello scraper, ove vi sono gli script e gli html devono essere posizionati
path_main_directory = '/Users/mattia/Desktop/Università/Data Science in Python/14) Web Scraping/Scraping-from-Agoda'

In [7]:
# Importare l'html del file in questione
with open(f"{path_main_directory}/html/html_final.txt","r") as f:
   html = f.read()

html_bs4 = BeautifulSoup(html,'html.parser')
proprietà = html_bs4.find_all("div",{"data-element-name":"PropertyCardBaseJacket"})


booking_dataframe = pd.DataFrame(columns=[
    "titolo",
    "recensione_voto_numerico",
    "recensione_voto_parola",
    "numero_recensioni",
    "date",
    "permanenza",
    "prezzo",
    "indirizzo",
    "valutazione"
])


for post in tqdm(proprietà):
   
   # DIV info proprietà
   header = post.find("header",{"data-element-name":"property-info-header"})
   
   # Titolo
   try: # Se non ha il titolo non è valida, passare alla successiva
      titolo = header.find("h3",{"data-selenium":"hotel-name"}).text
   except:
      continue 

   #  rating-container
   rating = header.find("div",{"data-testid":"rating-container"})
   if rating != None:
      rating = rating.find_all("svg",{"opacity":"1"})
      stelle = len(rating)
   else:
      stelle = np.nan

   
   indirizzo = header.find("div",{"data-selenium":"area-city"}).text
   indirizzo = indirizzo.replace("- Guarda sulla mappa","")
   
   # Prezzi e recensioni
   informazioni = post.find("div",{"data-element-name":"property-card-pricing-info"})
   
   ## Recensioni
   informazioni_recensioni = informazioni.find("div",{"class":"Box-sc-kv6pi1-0 hRUYUu PropertyCard__PricingHeadliner"})
  
   ### Questa div è molto caotica, bisogna cercare le tag p che contengono le recensioni
   
   for p in informazioni_recensioni.find_all("p"):
      
      spans = p.find_all("span")
      if  len(spans) > 0: # se ci sono più span nella tag p allora si tratta dei voti numerici o parola
         for span in spans:
            if "," in span.text: # se c'è una virgola è voto numerico
               recensione_voto_numerico = span.text
            else: # se non c'è la virgola è voto parola
               recensione_voto_parola = span.text
      else: # se non ci sono span è il numero di recensioni
        numero_recensioni = p.text


   prezzo = informazioni.find("div",{"data-element-name":"property-card-price"})
   prezzo = prezzo.find("div",{"data-element-name":"final-price"}).text


   # Crea il dizionario con i dati della singola proprietà
   informazioni_singola_proprietà = {  "titolo": titolo,
                                       "recensione_voto_numerico": recensione_voto_numerico,
                                       "recensione_voto_parola": recensione_voto_parola,
                                       "numero_recensioni": numero_recensioni,
                                       "date": "1 agosto - 2 agosto",
                                       "permanenza": "1 notte, 2 adulti",
                                       "prezzo": prezzo,
                                       "indirizzo": indirizzo,
                                       "valutazione": stelle,
                                     }
   
   # Aggiunge direttamente come nuova riga
   booking_dataframe.loc[len(booking_dataframe)] = informazioni_singola_proprietà


100%|██████████| 90/90 [00:00<00:00, 1003.21it/s]


In [4]:
booking_dataframe.to_csv(f"{path_main_directory}/agodaTerni.csv")

In [8]:
booking_dataframe.shape

(90, 9)

In [9]:
pd.set_option('display.max_rows', None)
booking_dataframe.sort_values(by="titolo")

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione
53,A L'Aquila per un sogno,"10,0",Eccezionale,1 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",111 €,"Terni, Terni",3
58,A casa di Alma,"9,4",Eccezionale,106 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",81 €,"Terni, Terni",3
62,Acqua Marmore,"9,0",Eccezionale,81 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",125 €,"Terni, Terni",4
66,Agriturismo Madre Terra,"8,8",Fantastico,89 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",138 €,"Narni, Narni",5
8,Agriturismo Tenuta Filippi,"8,9",Fantastico,254 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",123 €,"Narni, Narni",3
28,Alle Torri,"8,4",Fantastico,136 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",122 €,"Terni, Terni",4
80,Alloggio tra Terni e Narni,"9,5",Eccezionale,19 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",128 €,"Terni, Terni",3
17,Appartamenti Il Cerqueto 2,"8,4",Fantastico,160 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",74 €,"Terni, Terni",5
63,Appartamenti Natura e Terra Marmore,"9,1",Eccezionale,130 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",113 €,"Terni, Terni",4
65,Aria Marmore,"9,3",Eccezionale,36 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",153 €,"Terni, Terni",4
